# Conditional Random Fields

##### In NLP, it is a common task to extract words/ phrases of particular types from a given sentence or paragraph. For example, when performing analysis of a corpus of travel articles, we may want to know which travelling destination are mentioned in the articles, and how many articles are related to each of these travelling document.
##### Conditional Random Fields are discriminative model used for predicting the sequence of labels.<br><br> It uses information from the context through previous labels and thus helping the model in turn to make better prediction for unseen text/word etc.This technqiue can be very well used for Name Entity Recognition(NER), which is to give a accurate label to given word(incase of NLP).<br><br>For Example:<br> $\bullet\textit{"Shahjahan went to see Taj Mahal"}$ - in this example the $\textit{Taj Mahal}$ can be location representing a monument in Agra or can correspond to Tea Bags.<br>$\bullet\textit{"This Apple product is one in its segment"}$ - now here should the word Apple is to be associated with the Apple(Tech company) or a food item.<br> Hence, given a sequence of words identifying the correct sequence of labels is the problem to be solved and depending upon the problem finding the correct sequence of label, where label can be ${person, location, organization, etc}$ for instance. So, the CRF can solve these kind of problem which is a challenging task for the old traditional graphical models like $\underline{HMM}$ or $\underline{MaxEnt}$.<br><br>Let's try to understand through a bit more theory first and then I'll give a small implementation of CRF.I'll highlight few of the keywords discussed above first.

### Discriminative Model: 
##### In Machine Learning we have two different types of modelling technqiue.<br> $\bullet$ Discriminative - Logistic Regression, classifier based on Maximum Likelihood Estimation<br>$\bullet$ Generative - Naive Bayes is popular and simple probablistic classifier<br>

### CRF Model: A special case of undirected graphical model.
##### So, the basic crux of the CRF is to generate labels given huge amount of data where the input data is sequential. Also we consider previous context while making a prediction for given data point. <br><br>$\bullet$Let ${W}$ be the tokens in the document and ${w_{i}}$ be the corresponding word observed. <br>$\bullet$ ${y_i}$ be the hidden label.<br> The conditional distribtuion is modeled as: <br>\begin{equation*} \hat{y} = \underset{y}{\operatorname{argmax}} P(y|x)\end{equation*} <br><br> To model the appropriate behaviour of CRF we need a {feature function} which have the below parameters:<br> $\bullet$ Set of Input Vectors - W <br>$\bullet$ i - the word for which we want to predict label<br> $\bullet$ Label for data points ${w_{1:i-1}}$<br> $\bullet$ label of point $i$ in W <br> The feature functions are the key components of CRF and in our special case of linear-chain CRF, and the general form is: \begin{equation*}f_1(y_{i-1},y_{i},w_{1:N},i)\end{equation*} which looks at a pair of adjacent labels $y_{i−1}$, $y_i$, the whole input sequence $w_{1:N}$ , and the current position $i$. <br><br> For Ex: we can define a simple feature function which produces binary values: $\textit{1}$ for the current word is ${Taj Mahal}$, and if the current state ${y_n}$ is ${Monument}$.<br>Usage of such feature depends on the corresponding weight ${\lambda_1}$. If the ${\lambda_1 \gt 0}$ then label ${Monument}$ is preferred for above example otherwise CRF will try to avoid the label ${Monument}$ for the text ${TajMahal}$

### CRF vs HMM:
##### Although both are used to model sequential data, they are different algorithms.<br>Hidden Markov Models are generative, and give output by modeling the joint probability distribution whereeas Conditional Random Fields as mentioned above are discriminative, and model the conditional probability distribution. CRFs don’t rely on the independence assumption (labels are independent of each other), and avoid label bias. One way to look at it is that Hidden Markov Models are a very specific case of Conditional Random Fields, with constant transition probabilities used instead. HMMs are based on Naive Bayes, which we say can be derived from Logistic Regression, from which CRFs are derived.

### CRF Application:
##### $\bullet$ Part-of-Speech Tagging (POS).<br> $\bullet$ Named Entity Recognition.<br>$\bullet$ Image segmentation 

### Disadvantage:
##### Highly computational cost as well as complexity at the training stage of the algorithm. Re-training on new data is difficult.

### Minor Implementation:
#### Sequence labelling

In [1]:
import nltk
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from bs4 import BeautifulSoup as bs
from bs4.element import Tag
import codecs
import pycrfsuite
#nltk.download('averaged_perceptron_tagger')

In [85]:
def word2features(doc, i):
    #print(doc,i)
    word = doc[i][0]
    postag = doc[i][1]
    # Common features for all words
    features = [
        'bias',
        'word.lower=' + word.lower(),
        'word[-2:]=' + word[-2:],
        'word.isupper=%s' % word.isupper(),
        'word.isdigit=%s' % word.isdigit(),
        'postag=' + postag
                ]
    #print(features)
    # Features for words that are not at the beginning of a document
    if i > 0:
        word1 = doc[i-1][0]
        postag1 = doc[i-1][1]
        features.extend([
            '-1:word.lower=' + word1.lower(),
            '-1:word.isupper=%s' % word1.isupper(),
            '-1:word.isdigit=%s' % word1.isdigit(),
            '-1:postag=' + postag1
        ])
    else:
        # Indicate that it is the 'beginning of a document'
        features.append('<s>')

    # Features for words that are not at the end of a document
    if i < len(doc)-1:
        word1 = doc[i+1][0]
        postag1 = doc[i+1][1]
        features.extend([
            '+1:word.lower=' + word1.lower(),
            '+1:word.isupper=%s' % word1.isupper(),
            '+1:word.isdigit=%s' % word1.isdigit(),
            '+1:postag=' + postag1
        ])
    else:
        # Indicate that it is the 'end of a document'
        features.append('</s>')
    #print(features,"\n")
    return features

In [98]:
# extracting features in documents
def extract_features(doc):
    return [word2features(doc, i) for i in range(len(doc))]

In [99]:
# generating the list of labels for each document
def get_labels(doc):
    return [label for (token, postag, label) in doc]

In [100]:
# Read data file and parse the XML
with codecs.open("testFile.xml", "r", "utf-8") as infile:
    soup = bs(infile, "html5lib")
#print(soup.prettify(),"\n")
for elem in soup.find_all("document"):
    textContent = []
    # Loop through each child of the element under "textwithnamedentities"
    for c in elem.find("textwithnamedentities").children:
        if type(c) == Tag:
            if c.name == "namedentityintext":
                label = "N"  # part of a named entity
            else:
                label = "I"  # irrelevant word
            for w in c.text.split(" "):
                if len(w) > 0:
                    textContent.append((w, label))
    docs.append(textContent)
data = []
for i, doc in enumerate(docs):
    # fetching list of tokens in the document
    for _tuple in doc:
        tokens=_tuple[0]
    #POS tagging
    tagged = nltk.pos_tag(tokens)
    # creating a list of word-pos tag- label(N/I)
    data.append([(w, pos,label) for (w, label), (word, pos) in zip(doc, tagged)])
X = [extract_features(doc) for doc in data]
Y = [get_labels(doc) for doc in data]
#splitting data for training the data
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

trainer = pycrfsuite.Trainer(verbose=True)

# Submit training data to the trainer
for xseq, yseq in zip(X_train, Y_train):
    trainer.append(xseq, yseq)

# Set the parameters of the model
trainer.set_params({
    # coefficient for L1 penalty
    'c1': 0.2,
    # coefficient for L2 penalty
    'c2': 0.01,  
    # maximum number of iterations
    'max_iterations': 20,
    # whether to include transitions that
    # are possible, but not observed
    'feature.possible_transitions': True
})

# Provide a file name as a parameter to the train function, to save the model when training is finished
trainer.train('crf.model')
#print(X_test)
# Generate predictions
tagger = pycrfsuite.Tagger()
tagger.open('crf.model')
Y_pred = [tagger.tag(xseq) for xseq in X_test]

# Let's take a look at a random sample in the testing set
i = 2
for x, y in zip(Y_pred[i], [x[1].split("=")[1] for x in X_test[i]]):
    print("%s (%s)" % (y, x))

# Create a mapping of labels to indices
labels = {"N": 1, "I": 0}

# Convert the sequences of tags into a 1-dimensional array
predictions = np.array([labels[tag] for row in y_pred for tag in row])
truths = np.array([labels[tag] for row in y_test for tag in row])

# Print out the classification report
print(classification_report(truths, predictions,target_names=["I", "N"]))

Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 1
0....1....2....3....4....5....6....7....8....9....10
Number of features: 104
Seconds required: 0.004

L-BFGS optimization
c1: 0.200000
c2: 0.010000
num_memories: 6
max_iterations: 20
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

***** Iteration #1 *****
Loss: 79.321718
Feature norm: 1.000000
Error norm: 60.827283
Active features: 104
Line search trials: 1
Line search step: 0.006045
Seconds required for this iteration: 0.000

***** Iteration #2 *****
Loss: 52.427299
Feature norm: 1.611210
Error norm: 63.767277
Active features: 101
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.000

***** Iteration #3 *****
Loss: 30.925108
Feature norm: 2.883504
Error norm: 52.350005
Active features: 102
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.000

***